In [72]:
import numpy as np
import pandas as pd
import pybaseball
import seaborn as sns
import matplotlib.pyplot as plot
from deepdiff import DeepDiff
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register


In [73]:
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 100)

In [74]:
# Constants
KG_TO_LB = 0.453592
M_TO_IN = 0.0254


In [75]:
chadwick = chadwick_register(save=True)
chadwick = pd.DataFrame(chadwick)
download_lahman()
### Read infinity baseball data to DataFrames
# a table of all player biographical info and ids
people = pd.DataFrame(people())

# park id, name, alias, city, state, and country
parks = pd.DataFrame(parks())

# all star roster data: player, year, team, league, position
allstar = pd.DataFrame(all_star_full())

# each player's games played per position for each season
appearances = pd.DataFrame(appearances())

# batting stats by year, regular season
batting = pd.DataFrame(batting())

# batting stats by year, post season
batting_post = pd.DataFrame(batting_post())

# fielding stats by year
fielding = pd.DataFrame(fielding())

# games played in left, center, right field
fielding_of = pd.DataFrame(fielding_of())

# LF/CF/RF splits
fielding_of_split = pd.DataFrame(fielding_of_split())

# postseason fielding
fielding_post = pd.DataFrame(fielding_post())

# home game attendance by park by year
home_games = pd.DataFrame(home_games())

# historical player pitching stats
pitching = pd.DataFrame(pitching())

# postseason pitching stats
pitching_post = pd.DataFrame(pitching_post())

# playoff series winners and losers
series_post = pd.DataFrame(series_post())

# data on teams by year: record, division, stadium, attendance, etc
teams = pd.DataFrame(teams())

# current and historical franchises, whether they're still active, and their ids
teams_franchises = pd.DataFrame(teams_franchises())

# split season data for teams
teams_half = pd.DataFrame(teams_half())

# fangraphs batting since 2008
fangraphs_batting = pd.DataFrame(
    pybaseball.batting_stats_range(start_dt="2008-01-01", end_dt="2021-12-31")
)

# fangraphs pitching since 2008
fangraphs_pitching = pd.DataFrame(
    pybaseball.pitching_stats_range(start_dt="2008-01-01", end_dt="2021-12-31")
)

# fangraphs team pitching since 2008
fangraphs_team_pitching = pd.DataFrame(
    pybaseball.team_pitching(start_season="2008", end_season="2021")
)

# fangraphs team batting since 2008
fangraphs_team_batting = pd.DataFrame(
    pybaseball.team_batting(start_season="2008", end_season="2021")
)

# fangraphs team fielding since 2008
fangraphs_team_fielding = pd.DataFrame(
    pybaseball.team_fielding(start_season="2008", end_season="2021")
)

# bref pitching WAR
bwar_pitch = pd.DataFrame(bwar_pitch(return_all=True))

# bref batting WAR
bwar_bat = pd.DataFrame(bwar_bat(return_all=True))

/Users/bean/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pybaseball/league_batting_stats.py:64: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table = table.drop('', 1)


In [76]:
### Add BMI Calcs
# BMI Calculations
people["KG"] = people["weight"] * KG_TO_LB
people["meters"] = people["height"] * M_TO_IN
people["BMI"] = people["KG"] / people["meters"] ** 2
people["ratio"] = people["meters"] * people["BMI"]


## Assemble Monster DataFrame of Everything About Player Ever


In [77]:
# Let's change some of these column names to save ourselves some merging hassle
chadwick = chadwick.rename(
    columns={
        "name_last": "nameLast",
        "name_first": "nameFirst",
        "key_fangraphs": "fangraphsID",
        "key_bbref": "playerID",
        "key_retro": "retroID",
        "key_mlbam": "mlbID",
    }
)

# Merge chadwick into people
df = people.merge(
    chadwick,
    left_on=["playerID", "retroID", "nameLast", "nameFirst"],
    right_on=["playerID", "retroID", "nameLast", "nameFirst"],
    how="outer",
)

In [78]:
### Add BWAR Stats
#### Add BWAR Batting
##### BWAR Batting Prep
bwar_bat.columns
bwar_bat = bwar_bat.rename(columns={
    'player_ID': 'playerID',
    'mlb_ID': 'mlbID',
    'stint_ID': 'stint'
})

##### BWAR Batting Merge
df = df.merge(
    bwar_bat, left_on=["playerID", "mlbID"], right_on=["playerID", "mlbID"], how="outer"
)


In [79]:
#### Add BWAR Pitching
##### BWAR Pitching Prep

# merge bwar_pitch, check
bwar_pitch = bwar_pitch.rename(
    columns={"stint_ID": "stint", "mlb_ID": "mlbID", "player_ID": "playerID"}
)

##### BWAR Pitching Merge
print(df.columns.tolist())
df = df.merge(
    bwar_pitch,
    left_on=[
        "G",
        "WAA",
        "WAR",
        "WAR_rep",
        "age",
        "lg_ID",
        "mlbID",
        "oppRpG",
        "oppRpG_rep",
        "playerID",
        "pyth_exponent",
        "pyth_exponent_rep",
        "runs_above_avg",
        "runs_above_rep",
        "salary",
        "stint",
        "teamRpG",
        "team_ID",
        "waa_win_perc",
        "waa_win_perc_rep",
        "year_ID",
    ],
    right_on=[
        "G",
        "WAA",
        "WAR",
        "WAR_rep",
        "age",
        "lg_ID",
        "mlbID",
        "oppRpG",
        "oppRpG_rep",
        "playerID",
        "pyth_exponent",
        "pyth_exponent_rep",
        "runs_above_avg",
        "runs_above_rep",
        "salary",
        "stint",
        "teamRpG",
        "team_ID",
        "waa_win_perc",
        "waa_win_perc_rep",
        "year_ID",
    ],
    how="outer",
)

['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID', 'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common', 'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher', 'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p', 'runs_replacement', 'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off', 'waa_win_perc_def', 'waa_win_perc_rep', 'OPS_p

In [80]:
bwar_pitch.sample(10)

,name_common,age,mlbID,playerID,year_ID,team_ID,stint,lg_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg
23707,Syl Johnson,34.0,116630.0,johnssy01,1935,PHI,1,NL,37,18,524,436.0,88.0,79,93.510,0.000,NaN,-7.113,113,113.054,113.758,602.0,0.1166,-61.0,34.758,33.535,50.201,0.206,1.9368,5.20,NaN,4.74704,3.84069,1.846,0.5966,3.5742,0.1406,5.16525,1.923,0.4595,1.4824,127.972464,88.301
36920,Oliver Perez,22.0,424144.0,perezol01,2004,PIT,1,NL,30,30,588,588.0,0.0,71,104.221,4.044,NaN,-2.275,99,99.327,109.796,476.0,0.1063,-21.4,38.796,38.071,56.051,0.203,1.0000,5.72,321000.0,4.67092,3.40188,1.813,0.6399,4.1970,-0.1356,5.24637,1.923,0.4444,1.6604,143.358462,93.183
33176,Yellow Horse Morris,27.0,NaN,morriye01,1929,CAG,0,NNL,14,12,328,NaN,NaN,31,66.650,0.000,NaN,NaN,100,NaN,66.650,NaN,NaN,NaN,35.650,37.701,47.668,0.233,1.0000,4.86,NaN,5.27052,2.57759,1.799,0.7836,3.9704,-0.1600,6.13901,2.001,0.4243,1.0487,217.337500,52.161
7569,Spoon Carter,33.0,NaN,cartesp01,1936,PC,0,NN2,9,3,105,NaN,NaN,30,22.715,0.000,NaN,0.111,100,NaN,22.604,117.0,0.0618,1.8,-7.396,-6.878,-3.042,0.263,1.0000,-0.26,NaN,5.96508,6.72930,2.063,0.4381,-0.5571,-0.0528,6.44632,2.050,0.4603,0.3476,70.070370,18.919
39549,Alex Reyes,26.0,621052.0,reyesal02,2021,STL,1,NL,69,0,217,0.0,217.0,32,36.487,-2.379,4.352,2.930,92,92.233,32.770,168.0,0.0407,72.0,0.770,0.947,7.667,0.202,1.5593,0.64,900000.0,4.57192,4.55820,1.878,0.5014,0.0966,-0.1731,4.67269,1.885,0.4897,0.7123,121.553846,31.604
34107,Jeff Nelson,39.0,119704.0,nelsoje01,2006,CHW,1,AL,6,0,8,0.0,8.0,1,1.892,-0.141,NaN,-0.057,104,101.067,1.827,8.0,0.0017,-33.7,0.827,0.805,1.130,0.221,1.1200,0.11,NaN,4.92028,4.78611,1.911,0.5132,0.0792,-0.0022,4.97107,1.922,0.4951,0.0286,157.600000,1.576
1801,Homer Bailey,32.0,456701.0,baileho02,2018,CIN,1,NL,20,20,319,319.0,0.0,82,53.509,2.349,NaN,-0.952,103,101.684,57.767,384.0,0.0865,-11.0,-24.233,-22.838,-15.158,0.189,1.0000,-1.35,21000000.0,4.32337,5.46527,1.916,0.3896,-2.2080,-0.0830,4.77443,1.876,0.4536,0.9373,68.348611,49.211
41064,Randy Rosario,25.0,600968.0,rosarra01,2019,KCR,2,AL,6,0,11,0.0,11.0,1,2.348,-0.153,NaN,-0.030,103,102.429,2.279,11.0,0.0025,-12.0,1.279,1.281,1.678,0.221,1.3433,0.18,NaN,4.95606,4.74256,1.911,0.5210,0.1260,0.0126,5.02345,1.926,0.4935,0.0373,NaN,2.107
39199,Addison Reed,25.0,592665.0,reedad01,2014,ARI,1,NL,62,0,178,0.0,178.0,31,26.726,-2.015,NaN,0.728,102,102.254,24.524,167.0,0.0383,19.0,-6.476,-6.181,-1.802,0.175,1.6377,-0.51,538500.0,3.99035,4.09004,1.814,0.4888,-0.6944,-0.3619,4.06689,1.812,0.4914,0.5423,88.571429,24.800
29955,Brandon Maurer,23.0,543506.0,maurebr01,2014,SEA,1,AL,38,7,209,97.0,112.0,39,32.363,-0.721,NaN,1.096,95,94.073,28.736,227.0,0.0548,20.0,-10.264,-11.115,-3.939,0.184,0.7919,-0.55,504600.0,4.13265,4.42515,1.844,0.4685,-1.1970,-0.0526,4.29968,1.836,0.4818,0.7021,78.491667,28.257


In [81]:
#### Add Fielding_OF
##### Fielding_OF Prep
fielding_of.columns
fielding_of = fielding_of.rename(columns={"yearID": "year_ID"})
##### Fielding_OF Merge
df = df.merge(
    fielding_of,
    left_on=["playerID", "year_ID", "stint"],
    right_on=["playerID", "year_ID", "stint"],
    how="left",
)

In [82]:
#### Add Fielding
##### Fielding Check
fielding.columns
fielding = fielding.rename(columns={"yearID": "year_ID"})
##### Fielding Merge
df = df.merge(
    fielding,
    left_on=["playerID", "year_ID", "G", "stint"],
    right_on=["playerID", "year_ID", "G", "stint"],
    how="left",
)

In [34]:
### Meet the Monster
df.sample(25)  # works to here

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
54319,koplomi01,1976.0,8.0,30.0,USA,PA,Philadelphia,NaN,NaN,NaN,NaN,NaN,NaN,Mike,Koplove,Michael Paul,165.0,72.0,R,R,2001-09-06,2007-09-26,koplm001,koplomi01,74.842680,1.8288,22.377775,40.924475,407377.0,63.0,2001.0,2007.0,Mike Koplove,24.0,2001.0,ARI,1.0,NL,1.0,9.0,10.0,-0.30,0.00,0.00,0.0,0.0,0.0,0.0,NaN,0.0,0.00,0.14,0.00,-0.200,-0.200,-0.2,0.0,-0.0200,-0.02,-0.01,-0.02,-0.01,-0.02,0.0000,NaN,Y,4.71582,4.73359,0.08856,4.73013,1.897,1.897,0.4982,0.4982,0.5000,0.4997,-100.000000,0.350,0.455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARI,NL,P,0.0,30.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
154792,mcdanbo01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1942.0,KCM,0.0,NAL,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.179,8.823,NaN,NaN,1.3690,NaN,NaN,1.70,NaN,NaN,0.4758,NaN,NaN,4.96332,3.66212,NaN,5.50299,1.848,1.953,0.6369,NaN,NaN,0.4498,NaN,NaN,NaN,Booker McDaniel,4.0,177.0,NaN,NaN,18.0,26.823,0.000,NaN,NaN,100.0,NaN,26.823,NaN,NaN,NaN,13.012,0.215,1.0000,-0.1415,159.587500,25.534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2162,anderjo01,1873.0,12.0,14.0,Norway,NaN,Sarpsborg,1949.0,7.0,23.0,USA,MA,Worcester,John,Anderson,John Joseph,180.0,74.0,B,R,1894-09-08,1908-10-02,andej101,anderjo01,81.646560,1.8796,23.110376,43.438263,110244.0,1000232.0,1894.0,1908.0,John Anderson,28.0,1902.0,SLB,1.0,AL,543.0,126.0,NaN,-2.53,-0.72,0.00,-3.0,NaN,NaN,NaN,NaN,-3.0,-3.65,0.00,19.38,9.500,-9.900,-6.9,-6.7,-1.0200,-0.60,-0.64,0.89,-0.64,1.31,1.9100,3500.0,N,4.86686,4.92162,0.08903,4.76781,1.916,1.910,0.4923,0.4946,0.4949,0.4848,94.870143,185.443,198.858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,1.0,SLA,AL,1B,NaN,3351.0,1361.0,47.0,22.0,78.0,NaN,NaN,NaN,NaN,NaN
129256,bresnro01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111433.0,NaN,NaN,NaN,NaN,31.0,1910.0,STL,1.0,NL,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.611,0.360,NaN,NaN,0.0397,NaN,NaN,0.07,NaN,NaN,0.0287,NaN,NaN,4.04737,3.70337,NaN,4.30277,1.793,1.831,0.5397,NaN,NaN,0.4720,NaN,NaN,NaN,Roger Bresnahan,0.0,10.0,0.0,10.0,1.0,1.199,0.000,NaN,-0.189,98.0,98.000,1.360,15.0,0.0032,-59.0,0.344,0.176,1.0000,-0.0027,NaN,1.268,1.0,1.0,0.0,SLN,NL,P,0.0,9.0,0.0,3.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
129353,briceau01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,592169.0,NaN,NaN,NaN,NaN,27.0,2019.0,MIA,1.0,NL,NaN,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.192,1.030,NaN,NaN,0.1692,NaN,NaN,0.47,NaN,NaN,0.4082,555000.0,NaN,4.72014,4.67334,NaN,4.83750,1.893,1.903,0.5047,NaN,NaN,0.4883,NaN,NaN,NaN,Austin Brice,0.0,134.0,0.0,134.0,21.0,24.451,-1.658,NaN


# OPEN TRYOUTS ARE HERE


In [35]:
df_save = df

In [71]:
df_pitchers

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,22.0,2004.0,SFG,1.0,NL,0.0,11.0,10.7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,-0.01,0.00,-0.01,0.00,0.0,300000.0,Y,4.67092,4.67092,0.08651,4.67092,1.890,1.890,0.5000,0.5000,0.5,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SFN,NL,P,0.0,32.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,24.0,2006.0,CHC,1.0,NL,3.0,43.0,53.0,-0.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.46,0.0,-0.4,-0.4,-0.4,0.0,-0.04,-0.04,-0.01,-0.04,-0.01,-0.04,0.0,NaN,Y,4.85675,4.86675,0.09085,4.86457,1.912,1.913,0.4990,0.4990,0.5,0.4998,-100.000000,0.694,0.896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,25.0,2007.0,CHW,1.0,AL,0.0,2.0,32.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,387500.0,Y,4.85895,4.85895,0.08422,4.85895,1.912,1.912,0.5000,0.5000,0.5,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,26.0,2008.0,BOS,1.0,AL,1.0,5.0,48.7,-0.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.0,-0.2,-0.2,-0.2,0.0,-0.02,-0.02,0.00,-0.02,0.00,-0.02,0.0,403250.0,Y,4.67400,4.70400,0.08092,4.69650,1.893,1.894,0.4970,0.4970,0.5,0.4992,-100.000000,0.345,0.434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,27.0,2009.0,SEA,1.0,AL,0.0,3.0,71.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,419000.0,Y,4.79788,4.79788,0.08302,4.79788,1.905,1

In [36]:
df["BMI"] = round(df.BMI, 2)  # clean up our BMI decimals
df = df.dropna(subset=["BMI"])  # Drop anyone without a weight.
df_huskies_tryouts = df[df["weight"] >= 225]

df_huskies_tryouts = df_huskies_tryouts.sort_values("weight", ascending=False)

In [37]:
# weight of our heaviest starting lineup
df_huskies_tryouts.groupby("POS").weight.max().sum()

1892.0

In [70]:
df_huskies_tryouts

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
109928,youngwa01,1980.0,2.0,18.0,USA,MS,Hattiesburg,2015.0,9.0,19.0,USA,MS,Purvis,Walter,Young,Walter Ernest,320.0,77.0,L,R,2005-09-06,2005-10-02,younw001,youngwa01,145.14944,1.9558,37.95,74.214869,425488.0,2112.0,2005.0,2005.0,Walter Young,25.0,2005.0,BAL,1.0,AL,37.0,14.0,81.0,0.47,-0.12,0.19,0.0,0.0,0.0,0.0,0.0,0.0,-0.81,0.00,1.39,1.1,-0.3,-0.3,-0.8,-0.03,-0.03,-0.07,0.10,-0.07,0.10,0.13,NaN,N,4.69790,4.71718,0.08197,4.61808,1.895,1.890,0.4981,0.4981,0.4942,0.4900,115.029797,12.192,13.972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25063,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,30.0,2014.0,CIN,1.0,NL,0.0,34.0,34.7,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,-0.03,0.00,-0.03,0.00,0.00,NaN,Y,3.99035,3.99035,0.07088,3.99035,1.808,1.808,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25064,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,31.0,2015.0,CIN,1.0,NL,0.0,58.0,60.3,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,-0.01,0.00,-0.01,0.00,0.00,510000.0,Y,4.22222,4.22222,0.07613,4.22222,1.837,1.837,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25065,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,32.0,2016.0,CIN,1.0,NL,1.0,45.0,43.0,-0.28,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.00,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,-0.02,-0.01,-0.02,-0.01,0.00,525000.0,Y,4.49699,4.50010,0.08241,4.49938,1.870,1.870,0.4997,0.4997,0.5000,0.4999,-100.000000,0.332,0.427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIN,NL,P,0.0,129.0,1.0,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
25066,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,33.0,2017.0,TBR,1.0,AL,0.0,2.0,30.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0

In [38]:
# Probably easiest if we make ourselves a dataframe for every position, since we'll need to fill up a 26-man roster, which is parsed into several discrete roles that each require a particular number of players. 

# These role counts aren't codified, but teams have nearly always carried the same distribution of player-roles.

df_huskies_tryouts_P = df_huskies_tryouts[df_huskies_tryouts['POS'] == "P"]
df_huskies_tryouts_C = df_huskies_tryouts[df_huskies_tryouts['POS'] == "C"]
df_huskies_tryouts_1B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "1B"]
df_huskies_tryouts_2B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "2B"]
df_huskies_tryouts_3B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "3B"]
df_huskies_tryouts_SS = df_huskies_tryouts[df_huskies_tryouts['POS'] == "SS"]
df_huskies_tryouts_OF = df_huskies_tryouts[df_huskies_tryouts['POS'] == "OF"]


In [69]:
df_huskies_tryouts_P

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
25065,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,32.0,2016.0,CIN,1.0,NL,1.0,45.0,43.0,-0.28,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.0,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,-0.02,-0.01,-0.02,-0.01,0.0,525000.0,Y,4.49699,4.50010,0.08241,4.49938,1.870,1.870,0.4997,0.4997,0.5000,0.4999,-100.000000,0.332,0.427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIN,NL,P,0.0,129.0,1.0,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
11968,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,31.0,1938.0,NYG,1.0,NL,18.0,43.0,NaN,-1.13,-0.52,-0.24,0.0,NaN,NaN,NaN,NaN,0.0,0.00,2.07,0.0,0.2,0.2,0.2,0.0,0.01,0.02,-0.03,0.01,-0.03,0.02,0.0,NaN,Y,4.42789,4.42371,0.07970,4.41010,1.862,1.861,0.5004,0.5004,0.5000,0.4986,45.477489,6.116,6.277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,5.0,11.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
11967,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,30.0,1937.0,NYG,2.0,NL,0.0,4.0,NaN,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,NaN,Y,4.55686,4.55686,0.08283,4.55686,1.877,1.877,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,0.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
11969,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,32.0,1939.0,NYG,1.0,NL,12.0,31.0,NaN,0.28,-0.46,0.17,0.0,NaN,NaN,NaN,NaN,0.0,0.00,1.32,0.0,1.3,1.3,1.3,0.0,0.14,0.14,-0.02,0.14,-0.02,0.14,0.0,7500.0,Y,4.51438,4.47213,0.08040,4.45954,1.870,1.866,0.5044,0.5044,0.5000,0.4987,110.357093,4.168,4.425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,0.0,12.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
11970,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,33.0,1940.0,NYG,1.0,NL,10.0,41.0,NaN,-1.89,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,1.10,0.0,-0.8,-0.8,-0.8,0.0,-0.08,-0.08,-0.01,

We'll need:
- 5 starting pitchers
- 7 relievers
- 2 catchers
- 4 starting infielders
- 2 utility infielders
- 5 outfielders


In [39]:
huskies_P = (
    df_huskies_tryouts_P.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(13, "weight")
)  # 13 heaviest P

In [40]:
huskies_C = (
    df_huskies_tryouts_C.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest catchers

In [41]:
huskies_1B = (
    df_huskies_tryouts_1B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(1, "weight")
)  # heaviest 1B

In [42]:
huskies_2B = (
    df_huskies_tryouts_2B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest 2B

In [43]:
huskies_3B = (
    df_huskies_tryouts_3B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(1, "weight")
)  # 1 heaviest 3B

In [44]:
huskies_SS = (
    df_huskies_tryouts_SS.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest SS

In [45]:
huskies_OF = (
    df_huskies_tryouts_OF.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(5, "weight")
)  # 5 heaviest OF

In [46]:
huskies = pd.concat(
    [huskies_P, huskies_C, huskies_1B, huskies_2B, huskies_3B, huskies_SS, huskies_OF],
    ignore_index=True,
)

In [49]:
# BEHOLD. THE WONKAVILLE HUSKIES.
huskies.POS.value_counts().sum()

26

In [64]:
df = huskies.dropna(axis=1)

In [65]:
print(list(df.columns))

['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID', 'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common_x', 'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_defense', 'runs_position', 'runs_position_p', 'runs_replacement', 'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'pitcher', 'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off', 'waa_win_perc_def', 'waa_win_perc_rep', 'TOB_lg', 'TB_lg', 'teamID', 'lgID', 'POS', 'PO', 'A', 'E', 'DP']


In [66]:
df.runs_position_p.value_counts()

0.00    18
0.14     2
2.57     1
5.86     1
0.27     1
0.15     1
0.16     1
0.29     1
Name: runs_position_p, dtype: int64

In [50]:
# check to make sure everyone's best season is in here
# make sure we've got a rotation and relievers
# clean up columns
huskies.weight.describe()


count     26.000000
mean     271.423077
std       18.465477
min      230.000000
25%      260.000000
50%      275.000000
75%      281.500000
max      315.000000
Name: weight, dtype: float64

In [ ]:
roster_of = df_huskies_tryouts_OF.groupby("playerID", as_index=False)["WAR"].max()